# Training ```FFANN```

Let's augment the ```FFANN``` class in order to be able to train it.

This is still a work in progress, but at least it can "fit" the *XOR*-gate and a linear function.

In [1]:
import FeedForwardANN as FFANN
import numpy as np

In [2]:
class FFv2(FFANN.FFANN):
    pass
    
        



In [3]:
lin=FFANN.linearActivation()
sig=FFANN.sigmoidActivation()



In [4]:
# data_in=[[_] for _ in np.linspace(0,1,250)]
# data_out=[[2*_] for _ in np.linspace(0,1,250)]
# brain=FFv2(1,1,[3,2,1],[lin,lin,lin,lin,lin])
# brain.init_params(-1,1)


data_in=[[1,1],[0,0],[1,0],[0,1]]
data_out=[[0],[0],[1],[1]]
brain=FFv2(2,1,[3],[sig,lin])
brain.init_params(-1,1)

In [5]:
def Q_i(signal,target):
    return (signal-target)**2.

def dQds_i(signal,target):
    return 2.*(signal-target)

In [6]:
Q=FFANN.loss(Q_i, dQds_i, brain)
strategy=FFANN.VanillaSGD(brain,Q,alpha=1e-1)

In [7]:
brain.SGD(strategy, data_in, data_out, abs_tol=1e-2, rel_tol=1e-3, step_break=150,max_step=15000)

In [8]:
meanQ=0
maxQ=0
for i,_ in enumerate(data_in):
    Qi=Q(brain(_),data_out[i])
    print('point:',_,'\n'
       'target:',data_out[i],
         'FFANN gives:',brain(_),
         'loss:',Qi)
    print('')
    if Qi > maxQ:
        maxQ=Qi
    meanQ+=Q(brain(_),data_out[i])
    
print('mean loss:', meanQ/float(len(data_out)))
print('max loss:', maxQ)


point: [1, 1] 
target: [0] FFANN gives: [0.0005396913240642842] loss: 2.9126672527026023e-07

point: [0, 0] 
target: [0] FFANN gives: [0.0005866065546686894] loss: 3.4410724998027007e-07

point: [1, 0] 
target: [1] FFANN gives: [0.9999448118861333] loss: 3.045727912167506e-09

point: [0, 1] 
target: [1] FFANN gives: [0.9996648233956278] loss: 1.1234335611848601e-07

mean loss: 1.8769076482029598e-07
max loss: 3.4410724998027007e-07
